# Using Postgres to Create a Table to Record Information on Storms

In [1]:
"""Loading packages, connecting to the server and creating the database"""

import psycopg2
import io
from urllib import request
import csv

conn = psycopg2.connect(dbname="costa", user="costa")
cur = conn.cursor()
conn.autocommit = True
cur.execute('DROP DATABASE IF EXISTS storm')
cur.execute('CREATE DATABASE storm OWNER costa')

In [2]:
'''Setting up a group to modify to the database along with one user, e.g. people involved in production'''

#cur.execute('DROP GROUP IF EXISTS data_production')
#cur.execute('DROP USER IF EXISTS member_1')
#cur.execute('''
#CREATE GROUP data_production NOLOGIN;
#REVOKE ALL ON storm_data FROM data_production;
#GRANT SELECT, INSERT, UPDATE ON storm_data to data_production;
#CREATE USER member_1 IN GROUP data_production
#''')

In [3]:
'''Setting up a group to read the table only along with one user, e.g. analysts'''

#cur.execute('DROP GROUP IF EXISTS analysts')
#cur.execute('DROP USER IF EXISTS analyst_1')
#cur.execute('''
#CREATE GROUP analysts NOLOGIN;
#REVOKE ALL ON storm_data FROM analysts;
#GRANT SELECT ON storm_data to analysts;
#CREATE USER analyst_1 IN GROUP analysts
#''')

In [17]:
"""Creating the table and schema guided by inspecting the input data"""

conn = psycopg2.connect(dbname="storm", user="costa")
cur = conn.cursor()
conn.autocommit = True
cur.execute('DROP TABLE IF EXISTS storm_data')
cur.execute('''
CREATE TABLE storm_data(
fid VARCHAR(32),
datetime TIMESTAMP WITH TIME ZONE,
btid INTEGER,
name TEXT,
lat DECIMAL(3,1),
long DECIMAL(4,1),
wind_kts INTEGER,
pressure INTEGER,
cat VARCHAR(2),
basin VARCHAR(15),
shape_length DECIMAL(8,6)
)
''')

# verifying table schema
cur.execute('SELECT * FROM storm_data LIMIT 0')
print(cur.description)

(Column(name='fid', type_code=1043, display_size=None, internal_size=32, precision=None, scale=None, null_ok=None), Column(name='datetime_utc', type_code=1114, display_size=None, internal_size=8, precision=None, scale=None, null_ok=None), Column(name='btid', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='name', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None), Column(name='lat', type_code=1700, display_size=None, internal_size=3, precision=3, scale=1, null_ok=None), Column(name='long', type_code=1700, display_size=None, internal_size=4, precision=4, scale=1, null_ok=None), Column(name='wind_kts', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='pressure', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='cat', type_code=1043, display_size=None, internal_size=2, precision=No

In [18]:
"""Inserting the data into the table"""

with request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv') as response:
    next(response)
    reader = csv.reader(io.TextIOWrapper(response))
    for row in reader:
        later_elements = row[5:]
        first_element = row[0]
        # formatting the input data for the timestamp datatype
        datetime = row[1] + "-" + row[2] + "-" + row[3] + " " + row[4][:2] + ":" + row[4][2:4] + " " + row[4][-1]
        # putting all the data together in the order of the schema (to allow for fast and efficient addition of data 
        # in the future)
        updated_row = [first_element] + [datetime] + later_elements
        cur.execute("INSERT INTO storm_data VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", updated_row) 

In [21]:
"""Adding additional data to the table"""

with request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data_additional.csv') as response:
    cur.copy_expert('COPY storm_data FROM STDOUT WITH CSV HEADER', response)

In [12]:
"""Closing connection"""

conn.close()